## Demo of fmm Python API

In [1]:
from fmm import Network,NetworkGraph,FastMapMatch,FastMapMatchConfig,UBODT
from fmm import UBODTGenAlgorithm
#! pip2 install pyarrow
import pyarrow
import pyarrow.parquet as pq
import os
import pandas as pd

In [2]:

def getPoints(file_path):
    if os.path.isfile(file_path):
        text_file = open(file_path, "r")
        wkt = text_file.read()
        text_file.close()
    return wkt
def save_results(file_base_name, cpath, opath, indices, m_wkt, e_wkt):
  text_file = open(file_base_name + '-cpath.txt', "w")
  text_file.write(cpath)
  text_file.close()
  text_file = open(file_base_name + '-opath.txt', "w")
  text_file.write(opath)
  text_file.close()
  text_file = open(file_base_name + '-indices.txt', "w")
  text_file.write(indices)
  text_file.close()
  text_file = open(file_base_name + '-m_wkt.txt', "w")
  text_file.write(m_wkt)
  text_file.close()
  text_file = open(file_base_name + '-e_wkt.txt', "w")
  text_file.write(e_wkt)
  text_file.close()

### Load network data and graph

In [3]:
#network = Network("data/edges.shp")
# network = Network('../data/GPS.nosync/sonoma/edges.shp')
# print "Nodes {} edges {}".format(network.get_node_count(),network.get_edge_count())
# print(network)
# graph = NetworkGraph(network)
network = Network("../data/GPS.nosync/sonoma/edges.shp","fid", "u", "v")
print "Nodes {} edges {}".format(network.get_node_count(),network.get_edge_count())
graph = NetworkGraph(network)

Nodes 19594 edges 47319


### Precompute an UBODT file

**This step can be skipped if you already created one UBODT file.**

In [4]:
ubodt_gen = UBODTGenAlgorithm(network,graph)

In [6]:
#status = ubodt_gen.generate_ubodt("../data/GPS.nosync/_street_nodes_dual.txt", 4, binary=False, use_omp=True)
#print status

Status: success
Time takes 1965.51 seconds



### Load UBODT data

In [5]:
ubodt = UBODT.read_ubodt_csv("../data/GPS.nosync/sonoma/ubodt.txt")

### Create FMM model

In [6]:
model = FastMapMatch(network,graph,ubodt)

### Define FMM configuration

In [7]:
# k = 50
# radius = 100
# gps_error = 50
# k = 64
# radius = 10
# gps_error = 50
k = 30
radius = 1
gps_error = 1
fmm_config = FastMapMatchConfig(k,radius,gps_error)

### Run map matching

#### Match a single trajectory

In [10]:
#wkt = "LINESTRING(0.200812146892656 2.14088983050848,1.44262005649717 2.14879943502825,3.06408898305084 2.16066384180791,3.06408898305084 2.7103813559322,3.70872175141242 2.97930790960452,4.11606638418078 2.62337570621469)"

In [9]:
def get_sub_result(input_path, output_path, index):
  input_source = input_path + str(index) + '-points.txt'
  out_dir = output_path + 'sub-result-' + str(index)
  if (os.path.isfile(input_source)) & (not os.path.isdir(out_dir)):
    wkt = getPoints(input_source)
    result = model.match_wkt(wkt,fmm_config)
    if len(list(result.cpath)) > 0:
      #dir = output_path + 'sub-result-' + str(index)
      os.mkdir(out_dir)
      output_base = out_dir + '/' + str(index)
      save_results(output_base,str(list(result.cpath)), str(list(result.opath)),str(list(result.indices)),result.mgeom.export_wkt(),result.pgeom.export_wkt())
      print('index %d sucess!' % index)
    else:
      print('index %d failed!' % index)



In [8]:
def get_sub_result_from_df(df, output_path, index, start_time, end_time, mode, min_count, max_count):
  if ((int(df.loc[index]['count']) >= min_count) & (int(df.loc[index]['count']) <= max_count)):
    flag = False
    if mode == 0:
      flag = (bool(df.loc[index]['is_process']) == False) & (bool(df.loc[index]['is_success']) == False)
    if mode == 1:
      flag = (bool(df.loc[index]['is_process']) == True) & (bool(df.loc[index]['is_success']) == False)
    if mode == 2:
      flag = True
    if mode == 3:
      flag = (bool(df.loc[index]['is_process']) == False) & (bool(df.loc[index]['is_success']) == False) & (pd.Timestamp(df.loc[index]['start_time']) >= start_time) & (pd.Timestamp(df.loc[index]['end_time']) < end_time)
    if mode == 4:
      flag = (bool(df.loc[index]['is_process']) == True) & (bool(df.loc[index]['is_success']) == False) & (pd.Timestamp(df.loc[index]['start_time']) >= start_time) & (pd.Timestamp(df.loc[index]['end_time']) < end_time)
    if mode == 5:
      flag = (pd.Timestamp(df.loc[index]['start_time']) >= start_time) & (pd.Timestamp(df.loc[index]['end_time']) < end_time)
    if flag:
      out_dir = output_path + 'sub-result-' + str(index)
      df.loc[index]['is_process'] == True
      wkt = df.loc[index]['res']
      result = model.match_wkt(wkt,fmm_config)
      if len(list(result.cpath)) > 1:
        #dir = output_path + 'sub-result-' + str(index)
        os.mkdir(out_dir)
        output_base = out_dir + '/' + str(index)
        save_results(output_base,str(list(result.cpath)), str(list(result.opath)),str(list(result.indices)),result.mgeom.export_wkt(),result.pgeom.export_wkt())
        df.loc[index]['is_success'] = True
        print('index %d sucess!' % index)
      else:
        df.loc[index]['is_success'] = False
        print('index %d failed!' % index)

In [9]:
df = pd.read_csv('../data/GPS.nosync/Sonoma-2631-new-points/out2.csv')

In [11]:
for index in [4, 23, 187, 370, 630, 831, 1000, 1269, 1332,1396,1472]:
  get_sub_result_from_df(df, '../data/GPS.nosync/output/', index,0, 0, 2, 20, 1000)

/home/UFAD/xiongruoyang/.local/lib/python2.7/site-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


index 4 sucess!
index 23 sucess!
index 370 sucess!
index 630 sucess!
index 831 sucess!
index 1000 sucess!
index 1269 sucess!
index 1472 failed!


/home/UFAD/xiongruoyang/.local/lib/python2.7/site-packages/ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [33]:
n = len(df)
n

27190

In [13]:
t1 = pd.Timestamp(year = 2019,  month = 10, day = 25,
           hour = 21, minute = 59, second = 59, tz = 'US/Pacific')
t2 = pd.Timestamp(year = 2019,  month = 10, day = 26,
           hour = 6, minute = 0, second = 0, tz = 'US/Pacific')
t3 = pd.Timestamp(year = 2019,  month = 10, day = 26,
           hour = 12, minute = 0, second = 0, tz = 'US/Pacific')
t4 = pd.Timestamp(year = 2019,  month = 10, day = 26,
           hour = 21, minute = 59, second = 59, tz = 'US/Pacific')
t5 = pd.Timestamp(year = 2019,  month = 10, day = 27,
           hour = 6, minute = 0, second = 0, tz = 'US/Pacific')
t6 = pd.Timestamp(year = 2019,  month = 10, day = 27,
           hour = 12, minute = 0, second = 0, tz = 'US/Pacific')
t7 = pd.Timestamp(year = 2019,  month = 10, day = 27,
           hour = 21, minute = 59, second = 59, tz = 'US/Pacific')
t8 = pd.Timestamp(year = 2019,  month = 10, day = 28,
           hour = 6, minute = 0, second = 0, tz = 'US/Pacific')
n = len(df) 
for index in range(1, n - 1):
  get_sub_result_from_df(df, '../data/GPS.nosync/output/26-06-26-12/', index,t2, t3, 5, 5, 10000000)
# for index in range(1, n - 1):
#   get_sub_result_from_df(df, '../data/GPS.nosync/output/26-12-26-22/', index,t3, t4, 5, 5, 10000000)
# for index in range(1, n - 1):
#   get_sub_result_from_df(df, '../data/GPS.nosync/output/26-22-27-06/', index,t4, t5, 5, 5, 10000000)
# for index in range(1, n - 1):
#   get_sub_result_from_df(df, '../data/GPS.nosync/output/27-06-27-12/', index,t5, t6, 5, 5, 10000000)
# for index in range(1, n - 1):
#   get_sub_result_from_df(df, '../data/GPS.nosync/output/27-12-27-22/', index,t6, t7, 5, 5, 10000000)


/home/UFAD/xiongruoyang/.local/lib/python2.7/site-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/UFAD/xiongruoyang/.local/lib/python2.7/site-packages/ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


index 111 sucess!
index 122 failed!
index 265 failed!
index 329 sucess!
index 373 sucess!
index 388 failed!
index 424 sucess!
index 441 failed!
index 477 sucess!
index 536 failed!
index 704 failed!
index 779 failed!
index 866 sucess!
index 1243 failed!
index 1259 failed!
index 1408 failed!
index 1731 failed!
index 1759 failed!
index 1904 sucess!
index 1964 failed!
index 2028 sucess!
index 2041 sucess!
index 2207 failed!
index 2236 failed!
index 2237 sucess!
index 2290 failed!
index 2342 failed!
index 2370 failed!
index 2591 failed!
index 2733 sucess!
index 2738 failed!
index 2802 sucess!
index 2809 failed!
index 3253 failed!
index 3374 sucess!
index 3518 sucess!
index 3542 failed!
index 3552 failed!
index 3723 failed!
index 3868 sucess!
index 3929 sucess!
index 4125 sucess!
index 4126 failed!
index 4156 failed!
index 4175 failed!
index 4299 failed!
index 4377 sucess!
index 4396 failed!
index 4400 sucess!
index 4622 failed!
index 4870 failed!
index 4954 failed!
index 5055 failed!
index 

/home/UFAD/xiongruoyang/.local/lib/python2.7/site-packages/ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


index 14 failed!
index 15 failed!


/home/UFAD/xiongruoyang/.local/lib/python2.7/site-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


index 21 sucess!
index 33 failed!
index 36 failed!
index 59 failed!
index 100 failed!
index 161 failed!
index 215 failed!
index 241 failed!
index 251 failed!
index 254 failed!
index 262 failed!
index 277 sucess!
index 319 failed!
index 326 failed!
index 342 failed!
index 345 sucess!
index 367 failed!
index 369 failed!
index 404 failed!
index 406 failed!
index 414 failed!
index 434 failed!
index 461 sucess!
index 487 failed!
index 489 sucess!
index 570 sucess!
index 579 failed!
index 659 failed!
index 683 failed!
index 686 failed!
index 736 failed!
index 767 failed!
index 774 failed!
index 788 failed!
index 824 sucess!
index 847 failed!
index 853 sucess!
index 914 failed!
index 1041 failed!
index 1055 failed!
index 1079 failed!
index 1091 failed!
index 1092 sucess!
index 1105 failed!
index 1106 failed!
index 1107 failed!
index 1129 failed!
index 1207 sucess!
index 1225 failed!
index 1261 failed!
index 1264 failed!
index 1272 sucess!
index 1282 sucess!
index 1291 sucess!
index 1307 faile

/home/UFAD/xiongruoyang/.local/lib/python2.7/site-packages/ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


index 56 failed!


/home/UFAD/xiongruoyang/.local/lib/python2.7/site-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


index 233 failed!
index 889 sucess!
index 993 sucess!
index 994 sucess!
index 1018 sucess!
index 1112 failed!
index 1599 failed!
index 1745 sucess!
index 2035 sucess!
index 2337 sucess!
index 2338 sucess!
index 2780 failed!
index 2838 failed!
index 3295 sucess!
index 3349 sucess!
index 3524 failed!
index 3583 sucess!
index 3780 sucess!
index 4077 failed!
index 4515 sucess!
index 4634 failed!
index 4762 failed!
index 5072 sucess!
index 5178 failed!
index 5421 failed!
index 5540 sucess!
index 5912 sucess!
index 6018 sucess!
index 6326 failed!
index 6809 sucess!
index 7027 sucess!
index 7050 failed!
index 7070 sucess!
index 7136 sucess!
index 7466 sucess!
index 7484 sucess!
index 7576 failed!
index 7770 failed!
index 7902 failed!
index 8722 failed!
index 8821 sucess!
index 8826 failed!
index 9030 sucess!
index 9330 sucess!
index 9475 failed!
index 9492 sucess!
index 9696 sucess!
index 10624 sucess!
index 10714 sucess!
index 10909 sucess!
index 11001 sucess!
index 11151 sucess!
index 11340

In [42]:
df.loc[27181]

Unnamed: 0                                                27181
index                                                     27181
caid          ffec6163f3174bbd29df807720e3c10689592a1a186071...
start_time                            2019-10-27 15:33:39-07:00
end_time                              2019-10-27 15:33:39-07:00
count                                                         1
is_process                                                False
is_success                                                False
res                               LINESTRING(-122.4677 38.2828)
Name: 27181, dtype: object

In [11]:
for index in [4, 23, 187, 370, 630, 831, 1000, 1269, 1332,1396,1472]:
  get_sub_result('../data/GPS.nosync/points/', '../data/GPS.nosync/output/', index)

index 4 sucess!
index 23 sucess!
index 187 sucess!
index 370 sucess!
index 630 sucess!
index 831 sucess!
index 1000 sucess!
index 1269 sucess!
index 1332 sucess!
index 1396 sucess!
index 1472 sucess!


In [82]:
wkt = getPoints('data/GPS.nosync/points/4-points.txt')
#wkt = getPoints('data/GPS.nosync/output/00074014459977889ff22e04d309cbc68fb1720990d2d6a7251eb6c17fbf5f72-points.txt')

In [ ]:
result = model.match_wkt(wkt,fmm_config)
print "Matched path: ", list(result.cpath)
# print "Matched edge for each point: ", list(result.opath)
# print "Matched edge index ",list(result.indices)
# print "Matched geometry: ",result.mgeom.export_wkt()
# print "Matched point ", result.pgeom.export_wkt()

In [ ]:
len(list(result.cpath)) > 1

In [ ]:
save_results('data/GPS.nosync/output/0-00074',str(list(result.cpath)), str(list(result.opath)),str(list(result.indices)),result.mgeom.export_wkt(),result.pgeom.export_wkt())

##Print detailed match information

In [ ]:
candidates = []
for c in result.candidates:
    candidates.append((c.edge_id,c.source,c.target,c.error,c.length,c.offset,c.spdist,c.ep,c.tp))

In [ ]:
import pandas as pd
df = pd.DataFrame(candidates,
                  columns=["eid","source","target","error","length","offset","spdist","ep","tp"])

In [ ]:
df.head()

#### Match trajectories in a GPS file

In [ ]:
from fmm import GPSConfig,ResultConfig

In [ ]:
input_config = GPSConfig()
input_config.file = "../data/trips.csv"
input_config.id = "id"

In [ ]:
print input_config.to_string()

In [ ]:
result_config = ResultConfig()
result_config.file = "../data/mr.txt"
result_config.output_config.write_opath = True
print result_config.to_string()

In [ ]:
status = model.match_gps_file(input_config, result_config, fmm_config)

In [ ]:
print status

In [ ]:


filename = "../data/in_zone_grouped_evacuation_stops.parquet" 



df = pq.read_table(source=filename).to_pandas()

In [ ]:
print(df.loc[100])